**Static Huffman Coding implementation**

In [ ]:
#Class to create nodes to build huffman tree
class HuffmanNode:
    def __init__(self, char=None, frequency=0, left=None, right=None):
        self.char = char
        self.frequency = frequency
        self.left = left
        self.right = right

#Function to build frequency table which reads input text character by character and stores count of every unique character
def build_frequency_table(text):
    frequency_table = {}
    for char in text:
        frequency_table[char] = frequency_table.get(char, 0) + 1
    return frequency_table

"""Builds huffman tree from frequency table by,
creating node for each entry or character in table
sorting them according to frequencies"""
def build_huffman_tree(frequency_table):
    nodes = [HuffmanNode(char, freq) for char, freq in frequency_table.items()]

    while len(nodes) > 1:
        nodes.sort(key=lambda x: x.frequency)
        left = nodes.pop(0)
        right = nodes.pop(0)
        parent = HuffmanNode(frequency=left.frequency + right.frequency, left=left, right=right)
        nodes.append(parent)

    return nodes[0]


#It is used to map every character to codeword(binary representation) performing recursive calls
#If node is a leaf node add codeword to dictionary if it contain left node recursively traverse left sub-tree
def build_codewords_mapping(node, code="", mapping=None):
    if mapping is None:
        mapping = {}

    if node.char is not None:
        mapping[node.char] = code
    if node.left is not None:
        build_codewords_mapping(node.left, code + "0", mapping)
    if node.right is not None:
        build_codewords_mapping(node.right, code + "1", mapping)

    return mapping

#This is used to do character by character encoding
def encode(text, codewords_mapping):
    encoded_text = ""
    for char in text:
        encoded_text += codewords_mapping[char]
    return encoded_text

#This is used to do character by character decoding
def decode(encoded_text, huffman_tree):
    decoded_text = " "
    current_node = huffman_tree

    for bit in encoded_text:
        if bit == "0" and current_node.left:
            current_node = current_node.left
        elif bit == "1" and current_node.right:
            current_node = current_node.right

        if current_node.char:
            decoded_text += current_node.char
            current_node = huffman_tree

    return decoded_text

#Create frq_table,huffman_tree,get codeword mapping and encoded text
def huffman_compress(text):
    frequency_table = build_frequency_table(text)
    huffman_tree = build_huffman_tree(frequency_table)
    codewords_mapping = build_codewords_mapping(huffman_tree)
    encoded_text = encode(text, codewords_mapping)

    return encoded_text, huffman_tree

#Used to decode encoded text
def huffman_decompress(encoded_text, huffman_tree):
    decoded_text = decode(encoded_text, huffman_tree)
    return decoded_text

text_to_compress = "hello world"
compressed_text, huffman_tree = huffman_compress(text_to_compress)
print("Original Text:", text_to_compress)
print("Compressed Text:", compressed_text)
print("Decompressed Text:", huffman_decompress(compressed_text, huffman_tree))


Original Text: hello world
Compressed Text: 11101111101011000000111001010011
Decompressed Text:  hello world


In [ ]:
#Write encode text to file
def write_encoded_text_to_file(encoded_text, output_file):
    with open(output_file, 'wb') as file:
        byte_array = bytearray()
        for i in range(0, len(encoded_text), 8):
            byte = encoded_text[i:i + 8]
            if len(byte) < 8:
                byte += '0' * (8 - len(byte))
            byte_array.append(int(byte, 2))
        file.write(byte_array)

#Read text from file
def read_text_from_file(input_file):
    with open(input_file, 'rb') as file:
        byte_array = file.read()
        binary_string = ''.join(format(byte, '08b') for byte in byte_array)
        return binary_string

#Compress file
def compress_file(input_file, output_file):
    with open(input_file, 'rb') as file:
        text_to_compress = file.read()

    compressed_text, huffman_tree = huffman_compress(text_to_compress.decode('utf-8'))

    write_encoded_text_to_file(compressed_text, output_file)
    return huffman_tree

#Decompress file
def decompress_file(input_file, output_file, huffman_tree):
    encoded_text = read_text_from_file(input_file)
    decoded_text = huffman_decompress(encoded_text, huffman_tree)

    with open(output_file, 'wb') as file:
        file.write(decoded_text.encode('utf-8'))

# Example Usage for file compression and decompression:
input_file = 'original_file.txt'
compressed_file = 'compressed.txt'
decompressed_file = 'decompressed.txt'

# Compress file
huffman_tree = compress_file(input_file, compressed_file)

# Decompress file
decompress_file(compressed_file, decompressed_file, huffman_tree)

# Verify the result
original_file_size = os.path.getsize(input_file)
compressed_file_size = os.path.getsize(compressed_file)
decompressed_file_size = os.path.getsize(decompressed_file)

print("Original File Size:", original_file_size, "bytes")
print("Compressed File Size:", compressed_file_size, "bytes")
print("Decompressed File Size:", decompressed_file_size, "bytes")


Original File Size: 2220 bytes
Compressed File Size: 1300 bytes
Decompressed File Size: 2222 bytes
